### CoreBx_island_refac - Interpolate the North Core Banks DEMs onto rotated 1-m grid and save each as a .nc file.

Versioning jumped from v6 to refac

New invV5
* Files are switched to the "merged DEMs" that Jin-Si made, so the rapid iteration can occur.
* Box is re-adjusted to accomodate the whole island. The resulting array is huge, but manageble.

New in v2
* Now 4D maps, two made made during visit to Santa Cruz and two ftp'd from Andy
* Apr. 9 - changed to _v3 for Sep map
* Now does the interpolation without the loop
* Apr. 21 - moved origin to SE to accomodate curvature in NE end of island. Add 400 m to size of array.
* Watch file names, esp. underline (or not) after "1m_DEM"

New in v6
* Added maps through Sep 28 2020

New in refac  
* Added newest DEMs, but they have not been clipped.
* Code is now maintained in ../src/CoreBx, so paths to output files has been added.
* Changed exent of interpolation grid to fit inside bbox of raster arrays
* Moved make_grid to CoreBx_funcs
* Switched names for alongshore and offshore
* Switched to nearest-neighbor interpolation


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray
# from dask.distributed import LocalCluster
from scipy import interpolate, signal
from CoreBx_funcs import *
%matplotlib inline

In [2]:
# April 9, 2020: Replaced   "2019-09-12-13_1m_DEM_4D_crop.tif",\
# with _v3 and re-ran on my desktop

# May 4, 2020 - Changed to use Jin-Si's merged dems
# Dec 11, 2021 - Switched to unclipped candidate DEMs

#fdir = "C:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/merged_dems/"
#fdir = "D:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/clipped_dems/"

fdir = 'C:/crs/proj/2021_FloSupp_Release/clipfest/dems/'
fnames = (\
#'20181006_Ocracoke_Inlet_to_Ophelia_Inlet_1m_UTM18N_NAVD88_adj_cog.tif',\
'20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
#'20190908_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
'20190912-13_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
#'20190913_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
'20191011_Ocracoke_Inlet_to_Cape_Lookout_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
'20191126_Ocracoke_Inlet_to_Cape_Lookout_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
'20200208-9_Ocracoke_Inlet_to_Cape_Lookout_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
'20200508-9_Ocracoke_Inlet_to_Cape_Lookout_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
'20200802_Ocracoke_Inlet_to_Cape_LookoutNAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
'20200805-8_Ocracoke_Inlet_to_Cape_LookoutNAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
'20200928_Ocracoke_Inlet_to_Cape_LookoutNAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
'20210430_Ocracoke_Inlet_to_Cape_LookoutNAD83_2011_NAVD88_UTM18_1m_adj_cog.tif')

titles = ([\
#         "Oct 6 2018 post-Florence"\
         "Aug 30 2019 pre-Dorian",\
         "Sep 12-13 2019 post-Dorian",\
         "Oct 11 2019 lidar merge",\
         "Nov 26 2019 post-Nor'easter",\
         "Feb 8-9 2020",\
         "May 8-9 2020",\
         "Aug 2 2020 pre-Isaias",\
         "Aug 5- 2020 post-Isaias",\
         "Sep 28 2020 post-Teddy",\
         "May 30 2021"])

dates = ([\
         "2019-08-30",\
         "2019-09-12",\
         "2019-10-11",\
         "2019-11-26",\
         "2020-02-09",\
         "2020-04-09",\
         "2020-08-05",\
         "2020-08-09",\
         "2020-09-28",\
         "2021-05-30"])

# fnames = (\
#           "C:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/merged_dems/2019-08-30_1m_DEM_4D_crop2_m.tif",\
#           "C:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/merged_dems/2019-09-12-13_1m_DEM_4D_v3_m.tif",\
#           "C:/crs/proj/2019_DorianOBX/SfM_OBX_results/dems/NCB_20191011_DEM_1m_lidarMerge_NAD83_2011_UTM18N_NAVD88_crs.tif",\
#           "C:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/merged_dems/2019-11-26_1m_DEM_4D_crop_m.tif",\
#           "C:/crs/proj/2019_DorianOBX/SfM_OBX_results/dems/NCB_20200208-09_DEM_1m_4D_NAD83_2011_UTM18N_NAVD88_crs.tif",\
# #           "C:/crs/proj/2019_DorianOBX/SfM_OBX_results/dems/NCB_20200508-09_DEM_1m_4D_NAD83_2011_UTM18N_NAVD88_crs.tif",\
# #           "C:/crs/proj/2019_DorianOBX/SfM_OBX_results/dems/NCB_20200802_DEM_1m_4D_NAD83_2011_UTM18N_NAVD88_cog.tif",\
# #           "C:/crs/proj/2019_DorianOBX/SfM_OBX_results/dems/NCB_20200805-09_DEM_1m_4D_NAD83_2011_UTM18N_NAVD88_cog.tif",\
# #           "C:/crs/proj/2019_DorianOBX/SfM_OBX_results/dems/NCB_20200928_DEM_1m_4D_NAD83_2011_UTM18N_NAVD88_cog.tif")

# titles = ([\
#          "Aug 30 2019 pre-Dorian",\
#          "Sep 12-13 2019 post-Dorian",\
#          "Oct 11 2019 lidar merge",\
#          "Nov 26 2019 post-Nor'easter"])
# #          "Feb 8-9 2020",\
# #          "May 8-9 2020",\
# #          "Aug 2 2020 pre-Isaias",\
# #          "Aug 5-9 2020 post-Isaias",\
# #          "Sep 28 2020 post-Teddy"])

nf = len(fnames)
nft = len(titles)
print('Length of datasets',nf,nft)

#TODO - Consider remaking this base file
fill_fnames = ('EBK_201909_YesLidar_Comb_Extent_m.tif')
fill_titles = ('Sep_fill')

# optional median-filter smoothing of original maps
smooth = False
# kernal size...this should be an odd number >= dxy/0.1
ksize = 3

Length of datasets 10 10


In [3]:
# Read in a dict to define the rotated coordinate system
r = yaml2dict('island_box.yml')
print(r)

# Make a grid 
xu,yu,xrot,yrot,xcoords,ycoords = make_grid(**r)

ny,nx = np.shape(xu)
print('Size of grid:',ny,nx)

{'name': 'ncorebx_refac', 'e0': 378490.0, 'n0': 3855740.0, 'xlen': 36500.0, 'ylen': 1500.0, 'dxdy': 1.0, 'theta': 42.0}
make_grid: Shape of xrot, yrot:  (1500, 36500) (1500, 36500)
corners x, corners y]
[[ 378490.03700711 3855740.70613772]
 [ 405614.07999221 3880163.30413921]
 [ 404611.05321328 3881277.2782326 ]
 [ 377487.01022818 3856854.68023111]
 [ 378490.03700711 3855740.70613772]]
Saving to ncorebx_refac.csv
Size of grid: 1500 36500


In [ ]:
%%time
# output for rotated DEMs
dem_path ='C:/crs/proj/2019_DorianOBX/Dorian_paper_analyses/rotated_dems/'

dslist=[]
for i, fn in enumerate(fnames):
    iswarned = False
    fpath = fdir+fn
    print(i, fpath)

    # Old way: open the tif with XArray as a DataArray
    #     da = xr.open_rasterio(fpath)
    # New way: use rioxarray
    # The "masked" option puts in NaNs, but takes longer.
    da = rioxarray.open_rasterio( fpath, masked=True )

    print( np.shape(np.flipud(da['y'].values)), np.shape(da['x'].values), np.shape( np.flipud(da.values)) )
    x = da['x'].values
    y = np.flipud(da['y'].values)

    # Not sure how da.values got a singleton dimension, but squeeze gets rid of it.
    # However, make sure to squeeze before flipping
    z = np.flipud(np.squeeze(da.values))
    print(np.shape(x),np.shape(y),np.shape(z))

    if(smooth):
        # smooth with 2D running median
        zs = signal.medfilt2d(z, kernel_size=ksize)
    else:
        zs = z

    f = interpolate.RegularGridInterpolator( (y, x), zs, method='nearest')   

    # Array for interpolated elevations
    zi=np.NaN*np.ones((ny,nx))
        
    # this is the fast iteration, which only works when all of the source points fall inside the target box
    try:
        zi=f((yu,xu))

    # this is a slow iteration through all of the points, but allows us to skip ones that are outside
    except:
        if(not iswarned):
            print("Warning: using slow iteration.")
            iswarned = True
        for ij in np.ndindex(zi.shape):
            try:
                zi[ij]=f((yu[ij],xu[ij]))
            except:
                zi[ij]=np.NaN

    #dar = xr.DataArray(zi,dims=['Alongshore','Cross-shore'],coords={'Alongshore': ycoords, 'Cross-shore':xcoords })
    dar = xr.DataArray(zi,dims=['Cross-shore','Alongshore'],coords={'Cross-shore': ycoords, 'Alongshore' :xcoords })

    dar = dar.chunk()
    dslist.append(dar)

dsa = xr.concat(dslist, dim='map')
fn = dem_path+r['name']+'.nc'
dsa.to_netcdf(fn)

0 C:/crs/proj/2021_FloSupp_Release/clipfest/dems/20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif
(26230,) (29150,) (1, 26230, 29150)
(29150,) (26230,) (26230, 29150)
1 C:/crs/proj/2021_FloSupp_Release/clipfest/dems/20190912-13_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif


In [ ]:
%%time
# Read in the fill map and make netcdf files
fill_dir = "C:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/merged_dems/"

fn = fill_dir+fill_fnames
print(fn)

# open the tif with XArray as a DataArray
#daf = xr.open_rasterio(fn)
daf = rioxarray.open_rasterio( fn, masked=True )


print( np.shape(np.flipud(daf['y'].values)), np.shape(daf['x'].values), np.shape( np.flipud(daf.values)) )
x = daf['x'].values
y = np.flipud(daf['y'].values)

# Not sure how da.values got a singleton dimension, but squeeze gets rid of it.
# However, make sure to squeeze before flipping
z = np.flipud(np.squeeze(daf.values))
print(np.shape(x),np.shape(y),np.shape(z))

f = interpolate.RegularGridInterpolator( (y, x), z, method='nearest')   

# Array for interpolated elevations
zi=np.NaN*np.ones((ny,nx))

# this is a slow iteration through all of the points, but allows us to skip ones that are outside
# for ij in np.ndindex(zi.shape):
#     try:
#         zi[ij]=f((yu[ij],xu[ij]))
#     except:
#         zi[ij]=np.NaN

# this is the fast technique.
zi=f((yu,xu))

da = xr.DataArray(zi,dims=['Alongshore','Cross-shore'],coords={'Alongshore': ycoords, 'Cross-shore':xcoords })
da = da.chunk()

fno = dem_path+r['name']+'_Sep_fill.nc'
da.to_netcdf(fno)